In [1]:
from ReadData import ReadData
from Build_Model import BuildModelClassify
import tensorflow as tf
from PIL import Image
import os
from IPython.display import display
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import ast
import mtcnn
from mtcnn import MTCNN

import json
import csv

In [2]:
if tf.test.gpu_device_name():
    print('GPU available:', tf.test.gpu_device_name())
else:
    print("No GPU available. Please make sure you have TensorFlow configured to use GPU.")

GPU available: /device:GPU:0


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
build_model = BuildModelClassify()
model = build_model.build()
model.load_weights('D:/Cuong/HackAthon/code/train1/FUN.h5')

In [5]:
def normalize_faces(face_arrays_batch):
    normalized_faces_batch = []

    for face_arrays in face_arrays_batch:
        normalized_face_arrays = [face_array / 255.0 for face_array in face_arrays]
        normalized_faces_batch.append(normalized_face_arrays)

    return normalized_faces_batch

In [6]:
def switch_column(ix1,ix2, row):
    temp = row[ix1]
    row[ix1] = row[ix2]
    row[ix2] = temp

In [7]:
def number2str(output):
    output[3] = changeRace(output[3])
    output[4] = changeAge(output[4])
    output[5] = changeEmotion(output[5])
    output[6] = changeGender(output[6])
    output[7] = changeSkintone(output[7])
    output[8] = changeMasked(output[8])
    
def changeRace(value):
    value_map = {  0: 'Mongoloid',
                   1: 'Caucasian',
                   2: 'Negroid'}
    return value_map.get(value,value)
    
def changeAge(value):
    value_map = {  0: '20-30s',
                   1: '40-50s',
                   2: 'Kid',
                   3: 'Senior',
                   4: 'Teenager',
                   5: 'Baby'}
    return value_map.get(value,value)

def changeMasked(value):
    value_map = {  0: 'unmasked',
                   1: 'masked',}
    return value_map.get(value,value)

def changeSkintone(value):
    value_map = {  0: 'mid-light',
                   1: 'light',
                   2: 'mid-dark',
                   3: 'dark'}
    return value_map.get(value,value)

def changeEmotion(value):
    value_map = {  0: 'Happiness',
                   1: 'Neutral',
                   2: 'Sadness',
                   3: 'Anger',
                   4: 'Surprise',
                   5: 'Disgust',
                   6: 'Fear'}
    return value_map.get(value,value)
            
def changeGender(value):
    value_map = {  0: 'Female',
                   1: 'Male',}
    return value_map.get(value,value)    


In [8]:
# Specify the path to your JSON file
json_file_path = "D:/Cuong/HackAthon/public_test_and_submission_guidelines/file_name_to_image_id.json"  # Replace with the actual path
detector = MTCNN()
# Load the JSON file
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
# CSV file path
csv_file_path = 'D:/Cuong/HackAthon/PRIVATETEST/result/answer.csv'
public_test_path = 'D:/Cuong/HackAthon/PRIVATETEST/private_test_data/'
# Open the CSV file in write mode
with open(csv_file_path, 'a', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)
    
    # # Write the header row
    # csv_writer.writerow(['file_name', 'bbox', 'image_id', 'race', 'age', 'emotion', 'gender', 'skintone','masked'])
    start_processing = False
    for filename in os.listdir(public_test_path):
        if filename == 'private_test_img_3915.jpg':
            start_processing = True
        if start_processing and filename.endswith(".jpg"):
            image_file_path = os.path.join(public_test_path, filename)
            image_name = filename
            id = data.get(image_name, None)
            image = Image.open(image_file_path)
            image = image.convert('RGB')
            pixels = np.asarray(image)
            results = detector.detect_faces(pixels)
            for result in results:
                x1, y1, width, height = results[0]['box']
                # deal with negative pixel index
                x1, y1 = abs(x1), abs(y1)
                x2, y2 = x1 + width, y1 + height
                # extract the face
                face = pixels[y1:y2, x1:x2]
                # resize pixels to the model size
                face_image = Image.fromarray(face)
                face_image = face_image.resize((224,224))
                face_array = np.asarray(face_image) /255.0
                face_array_reshaped = np.expand_dims(face_array, axis=0)
                predict = model.predict(face_array_reshaped)

                argmax_outputs = [np.argmax(prediction, axis=1) for prediction in predict]
                # Concatenate the arrays and convert to a flattened list
                output = [image_name] + [result['box']] + [id] + list(np.concatenate(argmax_outputs))
                # Switch column
                switch_column(3,4,output)
                switch_column(5,7,output)
                switch_column(6,8,output)
                switch_column(7,8,output)
                number2str(output)
                csv_writer.writerow(output)
            image.close()


1/1 [==============================] - 0s 94ms/step
